# AdaptiveMD

## Example 2 - Running of Tasks

In [1]:
import sys, os

In [2]:
# stop RP from printing logs until severe
# verbose = os.environ.get('RADICAL_PILOT_VERBOSE', 'REPORT')
os.environ['RADICAL_PILOT_VERBOSE'] = 'ERROR'

In [3]:
from adaptivemd import (
    Project,
    Event, FunctionalEvent
)

from adaptivemd.engine.openmm import OpenMMEngine
from adaptivemd.analysis.pyemma import PyEMMAAnalysis

/Users/jan-hendrikprinz/anaconda/lib/python2.7/site-packages/radical/utils/atfork/stdlib_fixer.py:58: UserWarning: logging module already imported before fixup.
  warnings.warn('logging module already imported before fixup.')


Let's open our `test` project by its name. If you completed the previous example this should all work out of the box.

In [4]:
project = Project('test')

Open all connections to the `MongoDB` and `Session` so we can get started.

Let's see where we are. These numbers will depend on whether you run this notebook for the first time or just continue again. Unless you delete your project it will accumulate models and files over time, as is our ultimate goal.

In [5]:
print project.files
print project.generators
print project.models

<StoredBundle with 18 file(s) @ 0x120eb17d0>
<StoredBundle with 2 file(s) @ 0x120eb1790>
<StoredBundle with 0 file(s) @ 0x120eb1750>


Now restore our old ways to generate tasks by loading the previously used generators.

In [6]:
engine = project.generators['openmm']
modeller = project.generators['pyemma']
pdb_file = project.files['initial_pdb']

## Run simulations

Now we really start simulations. The general way to do so is to create a simulation task and then submit it to a cluster to be executed. A `Task` object is a general description of what should be done and boils down to staging some files to your working directory, executing a bash script and finally moving files back from your working directory to a shared storage. RP takes care of most of this very elegantly and hence a `Task` is designed somewhat to cover the capabilities but in a somehow simpler and more pythonic way.

For example there is a RPC Python Call Task that allows you to execute a function remotely and pull back the results. 

### Functional Events

We want to first look into a way to run python code asynchroneously in the project. For this, write a function that should be executed. Start with opening a scheduler or using an existing one (in the latter case you need to make sure that when it is executed - which can take a while - the scheduler still exists).

If the function should pause, write `yield {condition_to_continue}`. This will interrupt your script until the function you return will return `True` when called.

In [7]:
def strategy():
    # create a new scheduler
    local_scheduler = project.get_scheduler(cores=2)
    # run 10 trajs of length 100 in parallel
    tasks = local_scheduler.submit(project.new_ml_trajectory(
        length=100, number=10))
    # continue (all tasks need to be done)
    yield tasks.is_done()
    # close scheduler when job is done
    local_scheduler.exit()

turn a generator of your function use add `strategy()` and not `strategy` to the `FunctionalEvent`

In [8]:
ev = FunctionalEvent(strategy())

and execute the event inside your project

In [9]:
project.add_event(ev)

after some time you will have 10 more trajectories. Just like that.

In [10]:
print '# of files', len(project.trajectories)

# of files 1


### Tasks

To actually run simulations you need to have a scheduler (maybe a better name?). This instance can execute tasks or more precise you can use it to submit tasks which will be converted to ComputeUnitDescriptions and executed on the cluster previously chosen. 

In [8]:
scheduler = project.get_scheduler(cores=2)  # get the default scheduler using 2 cores

Now we are good to go and can run a first simulation

This works by creating a Trajectory object with a filename, a length and an initial frame. Then the engine will take this information and create a real trajectory with exactly this name, this initil frame and the given length.

Since this is such a common task you can also submit just a `Trajectory` without the need tp convert it to a `Task` first (which the engine can also do).

Out project can create new names automatically and so we want 4 new trajectories of length 100 and starting at the existing pdb_file we use to initialize the engine.

In [9]:
trajs = project.new_trajectory(pdb_file, 100, 4)

Let's submit and see

In [10]:
scheduler.submit(trajs)

Once the trajectories exist these objects will be saved to the database. It might be a little confusing to have objects before they exist, but this way you can actually work with these trajectories like referencing even before they exist.

This would allow to write now a function that triggers when the trajectory comes into existance. But we are not doing this right now.

Wait is dangerous since it is blocking and you cannot do anything until all tasks are finished. Normally you do not need it. Especially in interactive sessions.

In [11]:
scheduler.wait()

Look at all the files our project now contains.

In [12]:
print '# of files', len(project.files)

# of files 22


Great! That was easy (I hope you agree). 

Next we want to run a simple analysis.

In [13]:
t = modeller.execute(list(project.trajectories))

In [14]:
scheduler(t)

In [15]:
scheduler.wait()

Let's look at the model we generated

In [16]:
print project.models

<StoredBundle with 1 file(s) @ 0x120eb1750>


And pick some information

In [17]:
print project.models.last.data['msm']['P']

[[ 0.80434783  0.03429576  0.05531053  0.10604588  0.        ]
 [ 0.01791559  0.81122449  0.08288145  0.03996614  0.04801233]
 [ 0.04185879  0.12007304  0.83806817  0.          0.        ]
 [ 0.09418292  0.06794849  0.          0.7974359   0.04043269]
 [ 0.          0.12800292  0.          0.06340333  0.80859375]]


Next example will demonstrate on how to write a full adaptive loop

### Events

A new concept. Tasks are great and do work for us. But so far we needed to submit tasks ourselves. In adaptive simulations we want this to happen automagically. To help with some of this events exist. This are basically a task_generator coupled with conditions on when to be executed.

Let's write a little task generator (in essence a function that returns tasks)

In [9]:
def task_generator():
    return [
        engine.task_run_trajectory(traj) for traj in
        project.new_ml_trajectory(100, 4)]

In [10]:
task_generator()

Now create an event.

In [27]:
ev = Event().on(project.on_ntraj(range(20,22,2))).do(task_generator)

`.on` specifies when something should be executed. In our case when the project has a number of 20 trajectories. This is not yet the case so this event will not do anything unless we simulation more trajectories.

`.do` specifies the function to be called.

The concept is borrowed from event based languages like often used in JavaScript. 

You can build quite complex execution patterns with this. An event for example also knows when it is finished and this can be used as another trigger.

In [28]:
def hello():
    print 'DONE!!!'
    return []  # todo: allow for None here

finished = Event().on(ev.on_done).do(hello)

In [29]:
scheduler.add_event(ev)
scheduler.add_event(finished)

DONE!!!


All events and tasks run parallel or at least get submitted and queue for execution in parallel. RP takes care of the actual execution.

In [34]:
print '# of files', len(project.files)

# of files 34


So for now lets run more trajectories and schedule computation of models in regular intervals.

In [35]:
ev1 = Event().on(project.on_ntraj(range(30, 70, 4))).do(task_generator)
ev2 = Event().on(project.on_ntraj(38)).do(lambda: modeller.execute(list(project.trajectories))).repeat().until(ev1.on_done)
scheduler.add_event(ev1)
scheduler.add_event(ev2)

In [87]:
len(project.trajectories)

43

In [94]:
len(project.models)

0

`.repeat` means to redo the same task when the last is finished (it will just append an infinite list of conditions to keep on running).

`.until` specifies a termination condition. The event will not be executed once this condition is met. Makes most sense if you use `.repeat` or if the trigger condition and stopping should be independent. You might say, run 100 times unless you have a good enough model. 

In [92]:
print project.files

<StoredBundle with 70 file(s) @ 0x12056f3d0>


### Strategies (aka **the brain**)

The brain is just a collection of events. This makes it reuseable and easy to extend.

In [93]:
project.close()